 <p style="text-align: right;"> &#9989; Put your name here</p>

# Pre-Class Assignment: Practice

In todays pre-class assignment you are going to practice what we have learned.  

# Goals for today's pre-class assignment 

</p>

1. topic
1. Assignment Wrap-up

In [4]:
# Read data for this assignment
%matplotlib inline 
import matplotlib.pyplot as plt
from scipy.misc import imread, imsave
from urllib.request import urlopen

url1 = 'http://res.cloudinary.com/miles-extranet-dev/image/upload/ar_16:9,c_fill,w_1000,g_face,q_50/Michigan/migration_photos/G21696/G21696-msubeaumonttower01.jpg'
with urlopen(url1) as file:
    im1 = imread(file, mode='RGB')
url2 = 'http://msutoday.msu.edu/_/img/assets/2013/beaumont-spring-1.jpg'
with urlopen(url2) as file:
    im2 = imread(file, mode='RGB')
    
f, (ax1, ax2) = plt.subplots(1, 2,figsize=(20,10))
ax1.imshow(im1)
ax2.imshow(im2)

ImportError: cannot import name 'imread' from 'scipy.misc' (C:\Users\Owner\Desktop\XSEDE\see-segment\envs\lib\site-packages\scipy\misc\__init__.py)

----
# 1. Point Seleciton

In [5]:
# The following command will install mpld3 - Uncomment and run once.

!pip install -t ./packages mpld3 

  Created wheel for mpld3: filename=mpld3-0.5.5-py3-none-any.whl size=200886 sha256=92ce84322a8357effb323059fc276b9df28bb02b05463dc5731527cbdc4ccc16
  Stored in directory: c:\users\owner\appdata\local\pip\cache\wheels\d9\ed\51\ac5c5c9f2f8105b101482c066a31f9e71a9eef9c2fd6379bdd
Successfully built mpld3


In [6]:
#Add Packages folder to path (Do only once per run of notebook:
from sys import path
package_folder = './packages'
if not package_folder in path:
    print('adding packages')
    path.append(package_folder)
else:
    print('found packages folder')

adding packages


In [7]:
# Code snipit to make point seleciton function (REQUIRES mpld3 installed)
%matplotlib inline 
import matplotlib.pyplot as plt
import matplotlib.image as img
import sys

import mpld3
from mpld3 import plugins
mpld3.enable_notebook()

# PLOTS THE IMAGE IN THE NOTEBOOK
def plot(imgname):
    fig, ax = plt.subplots()
    im = img.imread(imgname)
    plt.imshow(im, origin='lower')
    return fig

# FUNCTION CALLED IN THE NOTEBOOK
def pickpoints(fig='', radius=4, color="white", x = 'x', y = 'y'):
    if not fig:
        fig = plt.gcf()
    plugins.connect(fig, Annotate(radius, color, x, y)) # color='htmlcolorname', radius=int
    plugins.connect(fig, plugins.MousePosition())

# FORMATS x AND y LISTS INTO SHORTER DECIMALS, SO THEY'RE NOT TOO LENGTHY
def cleanformat(var):
    varlist = []
    if type(var) == float:
        varlist = '{:05.2f}'.format(var)
    else:
        for i in range(len(var)):
            varlist.append('{:05.2f}'.format(var[i]))
    return varlist

# MAIN CLASS THAT CONTAINS JAVASCRIPT CODE TO CREATE CIRCLES AND DRAG CIRCLES  
class Annotate(plugins.PluginBase):
    """A plugin that creates points in a figure by clicking the mouse"""
   
    JAVASCRIPT = r"""
    mpld3.register_plugin("annotate", Annotate);
    Annotate.prototype = Object.create(mpld3.Plugin.prototype);
    Annotate.prototype.constructor = Annotate;
    Annotate.prototype.requiredProps = [];
    Annotate.prototype.defaultProps = {radius: 4, color: "white", x: 'x', y: 'y'};
    function Annotate(fig, props){
        mpld3.Plugin.call(this, fig, props);
    };

    Annotate.prototype.draw = function(){
        
        /// NECESSARY STARTUP VARIABLES ///
        
        var fig = this.fig;
        var ax = fig.axes;
        var dataset = [];
        var svg = d3.select(".mpld3-figure");   // existing svg element
        var radius = this.props.radius;
        var color = this.props.color;
        var x = this.props.x;
        var y = this.props.y;
        var ax = fig.axes[0];
        
        
        /// INDEXES HTML DOC TO PULL VALUES FOR x,y CALIBRATION ///
        var xcal = this.parent.axes[0].position[0];
        var ycal = this.parent.axes[0].position[1];
        console.log('x calibration: ' + xcal);
        console.log('y calibration: ' + ycal);
        
        var xcommand = x+" = []";
        IPython.notebook.kernel.execute(xcommand);
        var ycommand = y+" = []";
        IPython.notebook.kernel.execute(ycommand);
        
        
        ////////// CREATE POINT COMPONENT //////////
        
        var update_coords = function() {
        
            return function() {
                var pos = d3.mouse(this),
                    xpos = ax.x.invert(pos[0]),
                    ypos = ax.y.invert(pos[1]);
                    
                var newpoint = {
                    cx: pos[0] + xcal,
                    cy: pos[1] + ycal,
                    r: radius,
                    fill: color
                };
                dataset.push(newpoint);
                
                var circles = svg.selectAll("circle")
                    .data(dataset)
                    .enter()
                    .append("circle")
                    .attr(newpoint)
                    .call(drag);
                       
                var xcommand = x+".append("+xpos+")";
                IPython.notebook.kernel.execute(xcommand);
                console.log(xcommand);
                var ycommand = y+".append("+ypos+")";
                IPython.notebook.kernel.execute(ycommand);
                console.log(ycommand);
                   
            };
        }();
        ax.baseaxes
            .on("mousedown", update_coords);

        
        
        ////////// DRAG POINT COMPONENT //////////
        
        var drag = d3.behavior.drag()
            .on("dragstart", dragstarted)
            .on("drag", dragged)
            .on("dragend", dragended);
            
        function dragstarted(d) {
             d3.event.sourceEvent.stopPropagation();
             d3.select(this).classed("dragging", true);
        }
        
        function dragged(d) {
             d3.select(this).attr("cx", d3.event.x)
                            .attr("cy", d3.event.y);             
        }

        function dragended(d, i) {
             d3.event.sourceEvent.stopPropagation();
             d3.select(this).classed("dragging", false);
             var calib_cx = d3.select(this)[0][0].cx.animVal.value - xcal;
             var calib_cy = d3.select(this)[0][0].cy.animVal.value - ycal;
             var xcommand = x+"["+i+"] = "+ax.x.invert(calib_cx);
             var ycommand = y+"["+i+"] = "+ax.y.invert(calib_cy);
             IPython.notebook.kernel.execute(xcommand);
             IPython.notebook.kernel.execute(ycommand);
             console.log(xcommand);
             console.log(ycommand);
        }


    };"""

    def __init__(self, radius=4, color="white", x ='x', y ='y'):
        self.dict_ = {"type": "annotate",
                      "radius": radius,
                      "color": color,
                      "x": x,
                      "y": y};

In [8]:
fig = plt.figure(figsize=(9,6))
plt.imshow(im1)
pickpoints(color='cyan', radius=2, x='x1', y='y1')

NameError: name 'im1' is not defined

<Figure size 648x432 with 0 Axes>

In [9]:
fig = plt.figure(figsize=(9,6))
plt.imshow(im2)
pickpoints(color='cyan', radius=2, x='x2', y='y2')

NameError: name 'im2' is not defined

<Figure size 648x432 with 0 Axes>

----
# 1. Topic one

==TOPIC DESCRIPTION==

In [10]:
%matplotlib inline
from __future__ import division
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc as misc


from skimage import transform

# Here are some libraries you may need to use
%matplotlib inline
import matplotlib.pylab as plt

import sympy as sp
sp.init_printing()

import numpy as np

from ipywidgets import interact
import math



ModuleNotFoundError: No module named 'sympy'

In [11]:
plt.imshow(im2)

NameError: name 'im2' is not defined

In [12]:
im = im1
def affine_image(a1=0,s=1,tx=0,ty=0, alpha=1):
    theta = -a1/180  * math.pi
    
    dx = tx*im.shape[1]
    dy = ty*im.shape[0]
    S = np.matrix([[1/s,0,0], [0,1/s,0], [0,0,1]])
    T2 = np.matrix([[1,0,im.shape[1]/2], [0,1,im.shape[0]/2], [0,0,1]])
    T1 = np.matrix([[1,0,-im.shape[1]/2-dx], [0,1,-im.shape[0]/2-dy], [0,0,1]])
    R = np.matrix([[math.cos(theta),-math.sin(theta),0],[math.sin(theta), math.cos(theta),0],[0,0,1]])
    img = transform.warp(im, T2*S*R*T1);
    plt.imshow(im2);
    plt.imshow(img, alpha=alpha);

    plt.show();

NameError: name 'im1' is not defined

In [ ]:
interact(affine_image, a1=(-180,180), s=(0.001,5), tx=(-1.0,1.0), ty=(-1,1,0.1),alpha=(0.0,1.0)); ##TODO: Modify this line of code

In [ ]:
# Here are some libraries you may need to use
%matplotlib inline
import matplotlib.pylab as plt

import sympy as sp
sp.init_printing()

import numpy as np

from ipywidgets import interact
import math


In [ ]:
from urllib.request import urlopen
from scipy.misc import imread, imsave

url = 'http://res.cloudinary.com/miles-extranet-dev/image/upload/ar_16:9,c_fill,w_1000,g_face,q_50/Michigan/migration_photos/G21696/G21696-msubeaumonttower01.jpg'

with urlopen(url) as file:
    im = imread(file, mode='RGB')
    
#Show the image
plt.imshow(im);

In [ ]:
##Put your function here
from skimage import transform

def affine_image(a1=0,s=0.5,tx=0,ty=0,shx=0, shy=0):
    T = np.matrix([[1,0,0],[0,1,0],[0,0,1]])
    #Your transformations go here.
    
    img = transform.warp(im, T);
    plt.imshow(img);
    plt.show();

In [ ]:

##ANSWER##

def affine_image(a1=0,s=0.5,tx=0,ty=0, shx=0, shy=0):
    theta = -a1/180  * math.pi
    
    dx = tx*im.shape[1]
    dy = ty*im.shape[0]
    S = np.matrix([[1/s,0,0], [0,1/s,0], [0,0,1]])
    SH = np.matrix([[1,shx,0], [shy,1,0], [0,0,1]])
    T2 = np.matrix([[1,0,im.shape[1]/2], [0,1,im.shape[0]/2], [0,0,1]])
    T1 = np.matrix([[1,0,-im.shape[1]/2-dx], [0,1,-im.shape[0]/2-dy], [0,0,1]])
    R = np.matrix([[math.cos(theta),-math.sin(theta),0],[math.sin(theta), math.cos(theta),0],[0,0,1]])
    img = transform.warp(im, T2*S*R*SH*T1);
    plt.imshow(img);

    plt.show();
##ANSWER##

In [ ]:

interact(affine_image, a1=(-180,180), s=(0.001,5), tx=(-1,1,0.1), ty=(-1,1,0.1), shx = (-1,1,0.1), shy = (-1,1,0.1)); ##TODO: Modify this line of code

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo("NzysdpexqsM",width=640,height=360)

**Quesiton:** Ask a question that requires a written answer?

<font size=8 color="#009600">&#9998;</font> Do This - Erase the contents of this cell and replace it with your answer to the above question!  (double-click on this text to edit this cell, and hit shift+enter to save the text)

 &#9989; **To Do:** Task to be completed by student.

In [ ]:
%matplotlib inline
import matplotlib.pylab as plt
import numpy as np
import sympy as sp
sp.init_printing(use_unicode=True)

In [ ]:
#Example matrix in sympy
sp.Matrix([[1,0], [0,1]])

In [ ]:
#Example equation in sympy
#First define all of the symples to use in our equation
x,y,z,om = sp.symbols('x, y, z, \omega', negative=False)
pa = om*x**2+om*y**2+om*z**2
pa

In [ ]:
#Put your code here

Example matrix notation:

$$ 
\left[
\begin{matrix}
    1   & 0 & 4  \\
    0   & 2 & -2  \\
    0   & 1 & 2 
 \end{matrix}
\, \middle\vert \,
\begin{matrix}
-10 \\ 3 \\ 1
\end{matrix}
\right] 
$$

----
# 2. Assignment wrap-up

Please fill out the form that appears when you run the code below.  **You must completely fill this out in order to receive credit for the assignment!**

Direct Link: https://goo.gl/YcjHHB

In [ ]:
from IPython.display import HTML
HTML(
"""
<iframe 
	src="https://goo.gl/YcjHHB?embedded=true" 
	width="80%" 
	height="1200px" 
	frameborder="0" 
	marginheight="0" 
	marginwidth="0">
	Loading...
</iframe>
"""
)

---------
### Congratulations, you're done with your pre-class assignment!

&#169; Copyright 2018,  Michigan State University Board of Trustees